In [2]:
import os
import sys
sys.path.append("/Users/shashanks./Downloads/Installations/ddn/")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from ddn.pytorch.node import *
from scipy.linalg import block_diag
from torch.utils.data import Dataset, DataLoader
from bernstein import bernstein_coeff_order10_new

#### CUDA Initializations

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [4]:
num = 20
t_fin = 8.0
a_obs = 1.0
b_obs = 1.0

tot_time = np.linspace(0.0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
nvar = np.shape(P)[1]

In [5]:
x_obs_temp = np.hstack((-2.0, -0.79, 3.0, 4.0))
y_obs_temp = np.hstack((-2.0, 1.0, -0.80, 2.0))
num_obs = np.shape(x_obs_temp)[0]

x_obs = np.ones((num_obs, num)) * x_obs_temp[:, np.newaxis]
y_obs = np.ones((num_obs, num)) * y_obs_temp[:, np.newaxis]

In [6]:
A_obs = np.tile(P, (num_obs, 1))
A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
Q_smoothness = np.dot(Pddot.T, Pddot)

xT A x -> (1 x 10) x (10 x 10) x (10 x 1) => 1 x 1
xT A x -> (B x 10) x (10 x 10) x (10 x B) => torch.diag(B x B) => B x 1

In [7]:
class OPTNode(AbstractDeclarativeNode):
    def __init__(self, P, Pddot, A_eq, A_obs, Q_smoothness, x_obs, y_obs, num=20, nvar=11, num_obs=4, a_obs=1.0, b_obs=1.0, rho_obs=0.3, rho_eq=10.0, weight_smoothness=10, maxiter=300, eps=1e-7, num_tot=80):
        super().__init__()
        self.P = torch.tensor(P, dtype=torch.double).to(device)
        self.Pddot = torch.tensor(Pddot, dtype=torch.double).to(device)
        self.A_eq = torch.tensor(A_eq, dtype=torch.double).to(device)
        self.A_obs = torch.tensor(A_obs, dtype=torch.double).to(device)
        self.Q_smoothness = torch.tensor(Q_smoothness, dtype=torch.double).to(device)
        self.x_obs = torch.tensor(x_obs, dtype=torch.double).to(device)
        self.y_obs = torch.tensor(y_obs, dtype=torch.double).to(device)
        
        self.num = num
        self.eps = eps
        self.nvar = nvar        
        self.a_obs = a_obs
        self.b_obs = b_obs        
        self.rho_eq = rho_eq
        self.num_obs = num_obs
        self.maxiter = maxiter
        self.num_tot = num_tot
        self.rho_obs = rho_obs
        self.weight_smoothness = weight_smoothness
        
    def objective(self, b, lamda_x, lamda_y, y):     
        b = b.transpose(0, 1)
        y = y.transpose(0, 1)
        lamda_x = lamda_x.transpose(0, 1)
        lamda_y = lamda_y.transpose(0, 1)
        bx_eq_tensor, by_eq_tensor = torch.split(b, 6, dim=0)
        ones_tensor = torch.ones(self.num_tot, dtype=torch.double).to(device)

        c_x = y[0:self.nvar]
        c_y = y[self.nvar:2 * self.nvar]
        alpha_obs = y[2 * self.nvar: 2 * self.nvar + self.num_tot]
        d_obs = y[2 * self.nvar + self.num_tot:]

        cost_smoothness_x = 0.5 * self.weight_smoothness * torch.diag(torch.matmul(c_x.T, torch.matmul(self.Q_smoothness, c_x)))
        cost_smoothness_y = 0.5 * self.weight_smoothness * torch.diag(torch.matmul(c_y.T, torch.matmul(self.Q_smoothness, c_y)))

        temp_x_obs = d_obs * torch.cos(alpha_obs) * self.a_obs
        b_obs_x = self.x_obs.view(-1, 1) + temp_x_obs

        temp_y_obs = d_obs * torch.sin(alpha_obs) * self.b_obs
        b_obs_y = self.y_obs.view(-1, 1) + temp_y_obs

        cost_obs_x = 0.5 * self.rho_obs * (torch.sum((torch.matmul(self.A_obs, c_x) - b_obs_x) ** 2, axis=0))
        cost_obs_y = 0.5 * self.rho_obs * (torch.sum((torch.matmul(self.A_obs, c_y) - b_obs_y) ** 2, axis=0))
        cost_slack = self.rho_obs * torch.sum(torch.max(1 - d_obs, ones_tensor), axis=0)        
        #cost_slack = self.rho_obs * torch.sum(F.relu(1 - d_obs), axis=0)

        cost_eq_x = 0.5 * self.rho_eq * torch.sum((torch.matmul(self.A_eq, c_x) - bx_eq_tensor) ** 2, axis=0)
        cost_eq_y = 0.5 * self.rho_eq * torch.sum((torch.matmul(self. A_eq, c_y) - by_eq_tensor) ** 2, axis=0)
        
        # B x 1
        cost_x = cost_smoothness_x + cost_obs_x + cost_eq_x - torch.diag(a))
        cost_y = cost_smoothness_y + cost_obs_y + cost_eq_y - torch.diag(torch.matmul(lamda_y.transpose(0, 1), c_y))
        cost = cost_x + cost_y + self.eps * torch.sum(c_x ** 2, axis=0) + self.eps * torch.sum(c_y ** 2, axis=0) + self.eps * torch.sum(d_obs ** 2, axis=0) + self.eps * torch.sum(alpha_obs ** 2, axis=0) + cost_slack
        return cost
    
    def optimize(self, b, lamda_x, lamda_y):
        bx_eq_tensor, by_eq_tensor = torch.split(b, 6, dim=0)
        
        d_obs = torch.ones(self.num_obs, self.num, dtype=torch.double).to(device)
        alpha_obs = torch.zeros(self.num_obs, self.num, dtype=torch.double).to(device)
        ones_tensor = torch.ones((self.num_obs, self.num), dtype=torch.double).to(device)
        cost_smoothness = self.weight_smoothness * torch.matmul(self.Pddot.T, self.Pddot)
        cost = cost_smoothness + self.rho_obs * torch.matmul(self.A_obs.T, self.A_obs) + self.rho_eq * torch.matmul(self.A_eq.T, self.A_eq)

        for i in range(self.maxiter):
            temp_x_obs = d_obs * torch.cos(alpha_obs) * self.a_obs
            temp_y_obs = d_obs * torch.sin(alpha_obs) * self.b_obs

            b_obs_x = self.x_obs.view(num * num_obs) + temp_x_obs.view(num * num_obs)
            b_obs_y = self.y_obs.view(num * num_obs) + temp_y_obs.view(num * num_obs)

            lincost_x = -lamda_x - self.rho_obs * torch.matmul(self.A_obs.T, b_obs_x) - self.rho_eq * torch.matmul(self.A_eq.T, bx_eq_tensor)
            lincost_y = -lamda_y - self.rho_obs * torch.matmul(self.A_obs.T, b_obs_y) - self.rho_eq * torch.matmul(self.A_eq.T, by_eq_tensor)

            lincost_x = lincost_x.view(-1, 1)
            lincost_y = lincost_y.view(-1, 1)

            sol_x, _ = torch.solve(lincost_x, -cost)
            sol_y, _ = torch.solve(lincost_y, -cost)

            sol_x = sol_x.view(-1)
            sol_y = sol_y.view(-1)

            x = torch.matmul(self.P, sol_x)
            y = torch.matmul(self.P, sol_y)

            wc_alpha = x - self.x_obs
            ws_alpha = y - self.y_obs
            alpha_obs = torch.atan2(ws_alpha * self.a_obs, wc_alpha * self.b_obs)

            c1_d = self.rho_obs * (self.a_obs ** 2 * torch.cos(alpha_obs) ** 2 + self.b_obs ** 2 * torch.sin(alpha_obs) ** 2)
            c2_d = self.rho_obs * (self.a_obs * wc_alpha * torch.cos(alpha_obs) + self.b_obs * ws_alpha * torch.sin(alpha_obs))
            d_temp = c2_d / c1_d
            d_obs = torch.max(d_temp, ones_tensor)

            res_x_obs_vec = wc_alpha - self.a_obs * d_obs * torch.cos(alpha_obs)
            res_y_obs_vec = ws_alpha - self.b_obs * d_obs * torch.sin(alpha_obs)

            res_eq_x_vec = torch.matmul(self.A_eq, sol_x) - bx_eq_tensor
            res_eq_y_vec = torch.matmul(self.A_eq, sol_y) - by_eq_tensor

            lamda_x -= self.rho_obs * torch.matmul(self.A_obs.T, res_x_obs_vec.view(-1)) + self.rho_eq * torch.matmul(self.A_eq.T, res_eq_x_vec)
            lamda_y -= self.rho_obs * torch.matmul(self.A_obs.T, res_y_obs_vec.view(-1)) + self.rho_eq * torch.matmul(self.A_eq.T, res_eq_y_vec)

        sol = torch.cat([sol_x, sol_y, alpha_obs.view(-1), d_obs.view(-1)])
        return sol

#     def solve(self, b, lamda_x, lamda_y):
#         _, batch_size = b.size()
#         y = torch.zeros(batch_size, 182, dtype=torch.double).to(device)
#         for i in range(batch_size):
#             b_cur = b[:, i]
#             lamda_x_cur = lamda_x[:, i]
#             lamda_y_cur = lamda_y[:, i]
#             sol = self.optimize(b_cur, lamda_x_cur, lamda_y_cur)
#             y[i, :] = sol
#         return y.T, None

    def solve(self, b, lamda_x, lamda_y):
        batch_size, _ = b.size()
        b = b.transpose(0, 1)
        lamda_x = lamda_x.transpose(0, 1)
        lamda_y = lamda_y.transpose(0, 1)
        y = torch.zeros(batch_size, 182, dtype=torch.double).to(device)
        for i in range(batch_size):
            b_cur = b[:, i]
            lamda_x_cur = lamda_x[:, i]
            lamda_y_cur = lamda_y[:, i]
            sol = self.optimize(b_cur, lamda_x_cur, lamda_y_cur)
            y[i, :] = sol
        return y, None

In [8]:
opt_node = OPTNode(P, Pddot, A_eq, A_obs, Q_smoothness, x_obs, y_obs)

#### PyTorch Declarative Function

In [9]:
# class TrajQPFunction(torch.autograd.Function):
#     """Generic declarative autograd function.
#     Defines the forward and backward functions. Saves all inputs and outputs,
#     which may be memory-inefficient for the specific problem.
    
#     Assumptions:
#     * All inputs are PyTorch tensors
#     * All inputs have a single batch dimension (b, ...)
#     """
#     @staticmethod
#     def forward(ctx, problem, *inputs):
#         output, solve_ctx = torch.no_grad()(problem.solve)(*inputs)
#         ctx.save_for_backward(output, *inputs)
#         ctx.problem = problem
#         ctx.solve_ctx = solve_ctx
#         return output.clone()

#     @staticmethod
#     def backward(ctx, grad_output):
#         output, *inputs = ctx.saved_tensors
#         problem = ctx.problem
#         solve_ctx = ctx.solve_ctx
#         output.requires_grad = True
#         inputs = tuple(inputs)
#         grad_inputs = problem.gradient(*inputs, y=output, v=grad_output,
#             ctx=solve_ctx)
#         return (None, *grad_inputs)

#### TrajNet

In [10]:
b_new = torch.randn(10, 12, dtype=torch.double).to(device)
test_lamda_x = torch.zeros(10, 11, dtype=torch.double).to(device)
test_lamda_y = torch.zeros(10, 11, dtype=torch.double).to(device)

In [11]:
y, _ = opt_node.solve(b_new, test_lamda_x, test_lamda_y)
b_new.size(), test_lamda_x.size(), test_lamda_y.size(), y.size()

(torch.Size([10, 12]),
 torch.Size([10, 11]),
 torch.Size([10, 11]),
 torch.Size([10, 182]))

In [35]:
a1 = y[8][:11]
a2 = y[8][11:22]

In [39]:
torch.matmul(torch.tensor(P), a1)

torch.Size([20])

In [12]:
b_new.requires_grad = True
y.requires_grad = True
b_new.size(), y.size()

(torch.Size([10, 12]), torch.Size([10, 182]))

In [13]:
# out = opt_node.jacobian(b_new, test_lamda_x, test_lamda_y, y=y)
out1 = opt_node.gradient(b_new, test_lamda_x, test_lamda_y, y=y)
# out[0].size(), out[0][3].size(), out1[0].size()

In [14]:
out1[0].size()

torch.Size([10, 12])

In [15]:
out = opt_node.jacobian(b_new, test_lamda_x, test_lamda_y, y=y)
out[0].size()

AttributeError: 'tuple' object has no attribute 'size'

In [19]:
y_obj = opt_node.objective(b_new, test_lamda_x, test_lamda_y, y)

In [20]:
y_obj

tensor([ -50.4437, -106.5767,  -18.1562,  -60.7939,  -70.3207, -116.4888,
         -28.8257,  -57.2044,  -58.8530,  -59.2709], dtype=torch.float64,
       grad_fn=<AddBackward0>)